In [1]:
import os
from os import getcwd
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import random
import glob
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,GlobalAveragePooling2D, concatenate
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
import skimage.io
from keras import backend as K


In [2]:
def mini_block(filters, x):
    conv = tf.keras.layers.Conv2D(filters , kernel_size=(3, 3),padding = "same")(x)
    bn = tf.keras.layers.BatchNormalization()(conv)
    act = tf.keras.layers.ReLU()(bn)
    return act

def double_mini_block(filters, x):
    t = mini_block(filters, x)
    t = mini_block(filters, t)
    return t

def MFL_Block(f,inp):
    block1 = mini_block(f,inp)
    block2 = double_mini_block(f,inp)
    c1 =  concatenate([block1, block2])
    conv = tf.keras.layers.Conv2D(f , kernel_size=(1, 1))(c1)
    block3 = mini_block(f,conv)
    block4 = mini_block(f,inp)
    c2 = concatenate([block3, block4])
    conv = tf.keras.layers.Conv2D(f, kernel_size=(1, 1))(c2)
    return conv

def MFL_Net():
    inp = tf.keras.layers.Input(shape=(224,224,3))
    b1 = mini_block(16,inp)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(b1)
    p1 = MFL_Block(16,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p1)
    p2 = MFL_Block(32,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p2)
    p3 = MFL_Block(64,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p3)
    p4 = MFL_Block(128,pool)
    globavg = tf.keras.layers.GlobalAveragePooling2D()(p4)
    drop = tf.keras.layers.Dropout(rate=0.5)(globavg)
    out = tf.keras.layers.Dense(2, activation='softmax')(drop)
    model = tf.keras.Model(inputs=inp, outputs=out)
    model.compile(optimizer = Adam(lr=0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
    return model


In [3]:
model = MFL_Net()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 224, 224, 16) 0           batch_normalization[0][0]        
_______________________________________________________________________________________